# Part 1 

In [2]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import OllamaEmbeddings

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['ASTRA_DB_API_ENDPOINT'] = os.getenv('ASTRA_DB_API_ENDPOINT')
os.environ['ASTRA_DB_APPLICATION_TOKEN'] = os.getenv('ASTRA_DB_APPLICATION_TOKEN')

In [7]:
embedding = OllamaEmbeddings(model='nomic-embed-text')

vectorstore = AstraDBVectorStore(
    collection_name = 'test',
    embedding = embedding,
    token = os.getenv('ASTRA_DB_APPLICATION_TOKEN'),
    api_endpoint = os.getenv('ASTRA_DB_API_ENDPOINT')
)

print('Vector Store Initialized')

Vector Store Initialized


In [19]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [20]:
FILE_PATH = os.path.join(os.getcwd(), 'Files')

loader = PyPDFDirectoryLoader(FILE_PATH)
documents = loader.load()

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)

In [24]:
inserted_ids = vectorstore.add_documents(final_documents)

In [26]:
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 79 documents.


In [27]:
# Checks your collection to verify the documents are embedded.
print(vectorstore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': '159f19357fbe4ad2b555b04209cff5d4', 'content': 'allowing us to witness the power and beauty of reinforcement learning.\nDrawback of DeepSeek-R1-Zero Although DeepSeek-R1-Zero exhibits strong reasoning\ncapabilities and autonomously develops unexpected and powerful reasoning behaviors, it faces\nseveral issues. For instance, DeepSeek-R1-Zero struggles with challenges like poor readability,\nand language mixing. To make reasoning processes more readable and share them with the\nopen community, we explore DeepSeek-R1, a method that utilizes RL with human-friendly\ncold-start data.\n2.3. DeepSeek-R1: Reinforcement Learning with Cold Start\nInspired by the promising results of DeepSeek-R1-Zero, two natural questions arise: 1) Can\nreasoning performance be further improved or convergence accelerated by incorporating a small\namount of high-quality data as a cold start? 2) How can we train a user-friendly model that\nnot only produces clear and coherent Chains 

# Part 2

In [28]:
retriever = vectorstore.as_retriever(
    search_kwargs = {
        'k' : 5
    }
)

In [29]:
retriever.invoke('What is deepseek')

[Document(metadata={'source': 'f:\\My-Learning-Related-Stuffs\\New_Learning\\Generative AI\\DataStax-AstraDB\\Files\\2501.12948v1.pdf', 'page': 8}, page_content='models to generate detailed answers with reflection and verification, gathering DeepSeek-R1-\nZero outputs in a readable format, and refining the results through post-processing by human\nannotators.\nIn this work, we collect thousands of cold-start data to fine-tune the DeepSeek-V3-Base as\nthe starting point for RL. Compared to DeepSeek-R1-Zero, the advantages of cold start data\n9'),
 Document(metadata={'source': 'f:\\My-Learning-Related-Stuffs\\New_Learning\\Generative AI\\DataStax-AstraDB\\Files\\2501.12948v1.pdf', 'page': 2}, page_content='mixing. To address these issues and further enhance reasoning performance, we introduce\nDeepSeek-R1, which incorporates a small amount of cold-start data and a multi-stage training\npipeline. Specifically, we begin by collecting thousands of cold-start data to fine-tune the\nDeepSeek-

In [33]:
from langchain.chat_models import ChatOllama
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [35]:
prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

In [36]:
llm = ChatOllama(model='deepseek-r1:1.5b')

In [37]:
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | llm 
    | StrOutputParser()
)

In [38]:
response = chain.invoke('how deepseek r1 is trained')

In [39]:
response

"<think>\nAlright, I need to figure out what DeepSeek-R1 is trained with based on the provided context. Let me go through each document and section step by step.\n\nFirst document: This one talks about cold start data for a model. It mentions generating CoT data to fine-tune without the base model. So this shows that they're using specific types of answers from the target model (DeepSeek-R1) for training purposes.\n\nSecond document: This is about training templates. It says DeepSeek-R1-Zero uses a template that first produces reasoning processes followed by final answers. They mention avoiding content biases, so maybe their training data doesn't have any explicit instructions except what's necessary based on the task.\n\nThird document: This describes cold start data collection, focusing on generating detailed answers and using post-processing by humans to refine results. It also talks about reinforcement learning and retraining the reward model. So again, they're collecting data that

In [40]:
vectorstore.delete_collection()